In [17]:
import ndcctools.taskvine as vine
import json
import os

In [18]:
!ls data/parquet/10split

group_1.parquet   group_3.parquet  group_6.parquet  group_9.parquet
group_10.parquet  group_4.parquet  group_7.parquet
group_2.parquet   group_5.parquet  group_8.parquet


In [19]:
import pandas as pd
df = pd.DataFrame()
directory = 'data/parquet/10split'
for parquet in os.listdir(directory):
    if parquet.endswith('.parquet'):
        tempDf = pd.read_parquet(os.path.join(directory, parquet))
        df = pd.concat([df, tempDf])

df = df.reset_index(drop=True)
df   

,photo_url,latitude,longitude,taxon_id,quality_grade,observed_on,ancestry,name,group
0,http://inaturalist-open-data.s3.amazonaws.com/...,43.408011,-80.504497,56061.0,research,2018-05-18,48460/47126/211194/47125/47124/71275/47204/147...,Alliaria petiolata28,3
1,http://inaturalist-open-data.s3.amazonaws.com/...,56.031236,37.836835,56061.0,research,2020-06-02,48460/47126/211194/47125/47124/71275/47204/147...,Alliaria petiolata28,3
2,http://inaturalist-open-data.s3.amazonaws.com/...,40.492557,-74.417954,56061.0,research,2019-03-26,48460/47126/211194/47125/47124/71275/47204/147...,Alliaria petiolata28,3
3,http://inaturalist-open-data.s3.amazonaws.com/...,42.302962,-71.093989,56061.0,research,2021-04-30,48460/47126/211194/47125/47124/71275/47204/147...,Alliaria petiolata28,3
4,http://inaturalist-open-data.s3.amazonaws.com/...,39.325331,-82.117119,56061.0,research,2021-03-27,48460/47126/211194/47125/47124/71275/47204/147...,Alliaria petiolata28,3
...,...,...,...,...,...,...,...,...,...
99995,http://inaturalist-open-data.s3.amazonaws.com/...,51.116544,40.362378,356174.0,research,2023-06-15,48460/47126/211194/47125/47163/48073/48072/60254,Alisma × bjoerkqvistii26,9
99996,http://inaturalist-open-data.s3.amazonaws.com/...,-34.508966,139.946943,519246.0,research,2020-07-18,48460/1/47120/372739/47158/184884/47208/71130/...,Alleculini26,9
99997,http://inaturalist-open-data.s3.amazonaws.com/...,29.652299,-95.553950,519246.0,research,2020-04-26,48460/1/47120/372739/47158/184884/47208/71130/...,Alleculini26,9
99998,http://inaturalist-open-data.s3.amazonaws.com/...,-34.508966,139.946943,519246.0,research,2020-07-18,48460/1/47120/372739/47158/184884/47208/71130/...,Alleculini26,9


In [20]:
import asyncio
import aiohttp
from urllib.parse import urlsplit, urlunsplit

async def estimate_image_size(session, url):
    try:
        # Send a HEAD request to get headers only
        async with session.head(url) as response:
            # Check if the request was successful and contains Content-Length
            if response.status == 200 and 'Content-Length' in response.headers:
                size_in_bytes = int(response.headers['Content-Length'])
                size_in_kb = size_in_bytes / 1024
                return size_in_kb
            else:
                return None
    except Exception:
        return None

async def try_different_extensions(session, url):
    # Try the original URL first
    size_kb = await estimate_image_size(session, url)
    if size_kb is not None:
        return size_kb

    # If the original URL doesn't work, try different extensions
    extensions = ['.JPG', '.jpeg', '.JPEG', '.jpg']
    url_parts = urlsplit(url)
    
    for ext in extensions:
        # Replace the extension in the URL
        base, current_ext = url_parts.path.rsplit('.', 1)
        new_url = urlunsplit((url_parts.scheme, url_parts.netloc, f"{base}{ext}", url_parts.query, url_parts.fragment))
        print(f"Retrying with {new_url}...")
        
        size_kb = await estimate_image_size(session, new_url)
        if size_kb is not None:
            return size_kb

    # Return 0 if none of the extensions worked
    print(f"Could not determine size for {url}")
    return 0

async def get_total_size(urls):
    total_size_kb = 0
    async with aiohttp.ClientSession() as session:
        tasks = [try_different_extensions(session, url) for url in urls]
        sizes_kb = await asyncio.gather(*tasks)

    total_size_kb = sum(sizes_kb)
    total_size_mb = total_size_kb / 1024
    total_size_gb = total_size_mb / 1024
    return total_size_kb, total_size_mb, total_size_gb

# Running the asyncio loop
async def run(urls):
    total_size_kb, total_size_mb, total_size_gb = await get_total_size(urls)
    print(f"Total Size: {total_size_mb:.2f} MB ({total_size_gb:.2f} GB) ")

# Now call run with await
#await run(df['photo_url'])

In [21]:
import subprocess

def get_local_ip():
    try:
        # Run the 'hostname -I' command
        result = subprocess.run(['hostname', '-I'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        # Get the output, strip any extra spaces or newlines
        ip_addresses = result.stdout.strip()
        return ip_addresses
    except subprocess.CalledProcessError as e:
        return f"Error: {e}"

# Get and print the local IP addresses
local_ips = get_local_ip()
print(f"My local IP addresses are: {local_ips}")
print(local_ips.split()[0])
managerIp = local_ips.split()[0]

My local IP addresses are: 10.140.116.54 10.141.32.54 10.0.116.54
10.140.116.54


In [22]:
slurm_script = """#!/bin/bash
#SBATCH --job-name=taskvine           
#SBATCH --output=result.out           
#SBATCH --account=nirav
#SBATCH --partition=standard           
#SBATCH --nodes=1
#SBATCH --ntasks=2                    
#SBATCH --time=01:00:00               

# Load any necessary modules
module load python/3.9
source ~/.bashrc
conda init bash
conda init
conda activate taskvine

# Execute the Python script or command
vine_submit_workers -T slurm -p "--cpus-per-task=4 --time=1:00:00" -t 01:00:00 """+managerIp+""" 9124 2

"""

In [23]:
slurm_filename = "taskvine.slurm"
with open(slurm_filename, "w") as slurm_file:
    slurm_file.write(slurm_script)

print(f"SLURM script saved as {slurm_filename}")

SLURM script saved as taskvine.slurm


In [24]:
%%bash
sbatch taskvine.slurm
python bin/TaskvineLDAWT.py 

Submitted batch job 3432095
Listening on port 9124
Waiting for tasks to complete...
